In [21]:
! pip install geopandas pandas sqlalchemy psycopg2-binary openpyxl geoalchemy2 python-dotenv

In [1]:
from sqlalchemy import create_engine 
import geopandas as gpd
from shapely.ops import nearest_points

import os
from dotenv import load_dotenv

# Create Database Connection

In [2]:
PATH_TO_DOT_ENV = "../../.env"

DATABASE_TYPE = "postgresql"
DATABASE_HOST = "localhost"

In [3]:
load_dotenv(PATH_TO_DOT_ENV)

DATABASE_NAME = os.environ.get("DATABASE_NAME")
POSTGRES_USER = os.environ.get("POSTGRES_USER")
POSTGRES_PASSWORD = os.environ.get("POSTGRES_PASSWORD")
POSTGRES_HOST_PORT = os.environ.get("POSTGRES_HOST_PORT")
POSTGRES_CONTAINER_PORT = os.environ.get("POSTGRES_CONTAINER_PORT")

In [4]:
engine = create_engine(f"{DATABASE_TYPE}://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{DATABASE_HOST}:{POSTGRES_HOST_PORT}/{DATABASE_NAME}")

## Database Constants

In [5]:
CWEEDS_META_TABLE_NAME = "W_m"
CWEEDS_META_SUBDIVISION_TABLE = "W_ms"
SUBDIVISION_TABLE_NAME = "S"

In [6]:
subdivision_query = f"""select s.cid, s.geometry from "{SUBDIVISION_TABLE_NAME}" s"""
cweeds_mata_query = f"""select wm."climate_ID", wm.first_yr, wm.last_yr, wm.geometry from "{CWEEDS_META_TABLE_NAME}" wm"""

# Read station and division data

In [7]:
division_df = gpd.read_postgis(
    sql=subdivision_query, 
    con=engine,
    geom_col="geometry",
    crs="EPSG:4326"
)  
# division_ddf = dgpd.from_geopandas(division_df, npartitions=N_PARTISIONS)
# del division_df

In [8]:
cweeds_meta_df = gpd.read_postgis(
    sql=cweeds_mata_query, 
    con=engine,
    geom_col="geometry",
    crs="EPSG:4326"
)  
# cweeds_meta_ddf = dgpd.from_geopandas(cweeds_meta_df, npartitions=N_PARTISIONS)
# del cweeds_meta_df

# Find all the staions in division

In [9]:
stations_in_subdivision = gpd.sjoin(
    cweeds_meta_df,
    division_df,
    how="inner",
    predicate="within").drop("index_right", axis=1)

In [10]:
stations_in_subdivision.rename({
    'cid': 'division_id'
    },
    axis=1,
    inplace=True)

In [14]:
stations_in_subdivision

,climate_ID,first_yr,last_yr,geometry,division_id
0,3010010,2003,2017,POINT (-112.97000 54.28000),72
1,3010237,2003,2017,POINT (-112.28000 53.92000),72
2,3060406,2003,2017,POINT (-112.82000 54.78000),72
3,3050519,1998,2017,POINT (-115.55000 51.19000),72
4,3030525,2003,2017,POINT (-112.30000 49.80000),72
...,...,...,...,...,...
559,2100805,2005,2017,POINT (-139.84000 67.57000),0
560,2100935,2007,2017,POINT (-136.22000 66.98000),0
561,2101102,2005,2017,POINT (-132.73000 60.17000),0
562,2101201,2005,2017,POINT (-128.82000 60.12000),0


In [15]:
stations_in_subdivision.to_postgis(
    name=CWEEDS_META_SUBDIVISION_TABLE, 
    con=engine, 
    if_exists='replace', 
    index=False
)

In [16]:
stations_in_subdivision = stations_in_subdivision.groupby("division_id").agg(lambda group: set(group))

In [14]:
# stations_in_subdivision.apply(lambda row: len(row.climate_ID), axis=1).plot(kind='hist',bins=35,figsize=(25, 5))

In [17]:
divisions_with_station = stations_in_subdivision.index.to_list()
print(f"stations found {len(divisions_with_station)}")

stations found 19


# Find the nearest station for division with no stations

In [20]:
division_with_no_stations = division_df[~division_df["cid"].isin(divisions_with_station)]
division_with_no_stations

,cid,geometry


# if found divisions with no stations run below code 

In [17]:
def get_nearest_station_id(division):
    divison_id = division.FEDUID
    division_geomaetry = division.geometry
    _, nearest_point = nearest_points(division_geomaetry.centroid, cweeds_meta_df.geometry.unary_union)
    station_id = cweeds_meta_df.loc[cweeds_meta_df['geometry'] == nearest_point].climate_ID.values[0]
    return station_id

In [18]:
# division_with_no_stations['climate_ID'] = division_with_no_stations.apply(get_nearest_station_id, axis=1)

In [19]:
# division_with_no_stations[['FEDUID','climate_ID']].to_sql(name=TABLE_NAME, con=engine, if_exists='append', index=False)

# Add Primary Key to Table

In [23]:
with engine.connect() as con:
    con.execute(f'ALTER TABLE "{CWEEDS_META_SUBDIVISION_TABLE}" ADD PRIMARY KEY ("division_id", "climate_ID");')